In [0]:
%pip install mlflow-extensions
%pip install -U mlflow
%restart_python

In [0]:
from mlflow_extensions.databricks.prebuilt import prebuilt
from mlflow_extensions.databricks.deploy.ez_deploy import EzDeploy

deployer = EzDeploy(
  config=prebuilt.vision.vllm.PHI_3_5_VISION_INSTRUCT_12K,
  registered_model_name="shm.default.phi_3_5_vision_instruct_12k"
)

In [0]:
deployer.download()

In [0]:
deployer.register()

In [0]:
endpoint_name = "sri_phi_3_5_vision_instruct_vllm"
deployer.deploy(endpoint_name)

WAIT FOR MODEL TO DEPLOY

In [0]:
import json, requests, os
input_example = {"inputs":{"messages":[{"role": "user", 
    "content": [
                {"type": "text", "text": "Is the image indoors or outdoors?"},
                {
                    "type": "image_url",
                    "image_url": {
                      "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    },
                },
            ],
    }]}}
url = 'https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/sri_phi_3_5_vision_instruct_vllm/invocations'
headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
data_json = json.dumps(input_example, allow_nan=True)
response = requests.request(method='POST', headers=headers, url=url, data=data_json)

In [0]:
response.content

In [0]:

from mlflow_extensions.serving.compat.openai import OpenAI
from mlflow.utils.databricks_utils import get_databricks_host_creds

workspace_host = spark.conf.get("spark.databricks.workspaceUrl")
endpoint_name = f"https://{workspace_host}/serving-endpoints/{endpoint_name}/invocations"
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()





# COMMAND ----------

client = OpenAI(
  base_url=endpoint_name,
  api_key=token
)

# COMMAND ----------

# Try this if you want to do guided decoding
# from pydantic import BaseModel

# class ExpectedJson(BaseModel):
#     outside: bool
#     inside: bool
#     boardwalk: bool
#     grass: bool

# while True:
response = client.chat.completions.create(
  model='Phi-3.5-vision-instruct',
  messages=[
    {"role": "user", 
    "content": [
                {"type": "text", "text": "Is the image indoors or outdoors?"},
                {
                    "type": "image_url",
                    "image_url": {
                      "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    },
                },
            ],
    }
  ],
  # extra_body={
  #   "guided_choice": ["outside", "indoors"]
  # }
  # extra_body={
  #   "guided_json": ExpectedJson.schema()
  # }
)

response.choices[0].message.content.strip()

# COMMAND ----------

